In [24]:
import xgboost as xgb
import pandas as pd
import numpy as np
import pickle as pkl
from sklearn.model_selection import train_test_split

In [14]:
models_path = '../../models/'
submit_path = '../../submit/'
raw_data_path = '../../data/raw/'
processed_data_path = '../../data/processed/'

# 0 - Read preprocessed files

In [15]:
df_train = pkl.load(open(processed_data_path + "train_baseline.pkl", "rb" ) )

In [17]:
df_test = pkl.load(open(processed_data_path + "test_baseline.pkl", "rb" ) )

# I - Check dataframes

In [18]:
df_train.shape

(4459, 4736)

In [19]:
df_train.head()

,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
ID,,,,,,,,,,,,,,,,,,,,,
000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,2200000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,2000000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [20]:
df_test.shape

(49342, 4735)

In [21]:
df_test.head()

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
ID,,,,,,,,,,,,,,,,,,,,,
000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# II - Split train test

In [22]:
y_train = np.log1p(df_train["target"].values)
df_train = df_train.drop(['target'], axis = 1)
print("y_train shape: {}".format(y_train.shape))

y_train shape: (4459,)


# III - Define model

In [27]:
params_xgb = {"objective": "reg:linear",
              "eval_metric": "rmse",
              "eta": 0.01,
              "max_depth": 15,
              "subsample": 0.70,
              "colsample_bytree": 0.5,
              'alpha':0,
              "seed": 1111,
              "n_jobs": -1
             }

num_trees = 2000
early_stopping_rounds = 30
verbose_eval = 10

nfold = 3 # CV only

## Cross validation

In [28]:
xgb.cv(params_xgb, xgb.DMatrix(df_train,y_train), num_boost_round=num_trees, nfold=nfold, verbose_eval=verbose_eval)

[0]	train-rmse:9.58956e+06+29988.9	test-rmse:9.69229e+06+82383
[1]	train-rmse:9.08848e+06+18415	test-rmse:9.31762e+06+100311
[2]	train-rmse:8.65627e+06+16356.9	test-rmse:8.98123e+06+115767
[3]	train-rmse:8.25371e+06+14406.3	test-rmse:8.68769e+06+107709
[4]	train-rmse:7.89326e+06+12216.6	test-rmse:8.44395e+06+115665
[5]	train-rmse:7.57624e+06+15122.3	test-rmse:8.23351e+06+115948
[6]	train-rmse:7.31761e+06+19176.7	test-rmse:8.06523e+06+113287
[7]	train-rmse:7.0719e+06+17778.8	test-rmse:7.91243e+06+108750
[8]	train-rmse:6.85881e+06+16222.1	test-rmse:7.78816e+06+118026
[9]	train-rmse:6.65657e+06+17022.2	test-rmse:7.68906e+06+108625
[10]	train-rmse:6.48128e+06+16004.3	test-rmse:7.60015e+06+113301
[11]	train-rmse:6.31973e+06+28510.1	test-rmse:7.52808e+06+107336
[12]	train-rmse:6.17316e+06+32367.8	test-rmse:7.46545e+06+102145
[13]	train-rmse:6.03449e+06+34489.6	test-rmse:7.41533e+06+96665.8
[14]	train-rmse:5.90759e+06+43887.1	test-rmse:7.37444e+06+99859.4
[15]	train-rmse:5.80003e+06+59016.2	t

[127]	train-rmse:2.96995e+06+57204.7	test-rmse:7.39612e+06+116253
[128]	train-rmse:2.96236e+06+60784.1	test-rmse:7.39803e+06+115207
[129]	train-rmse:2.95543e+06+62658.3	test-rmse:7.39822e+06+112689
[130]	train-rmse:2.94524e+06+57502.5	test-rmse:7.39762e+06+112231
[131]	train-rmse:2.92849e+06+55673.7	test-rmse:7.39796e+06+109679
[132]	train-rmse:2.92166e+06+54093.1	test-rmse:7.39854e+06+108967
[133]	train-rmse:2.91675e+06+52789.5	test-rmse:7.40104e+06+106900
[134]	train-rmse:2.90682e+06+48880	test-rmse:7.40325e+06+105181
[135]	train-rmse:2.90017e+06+51453.2	test-rmse:7.40442e+06+103399
[136]	train-rmse:2.89151e+06+47518.1	test-rmse:7.40694e+06+105407
[137]	train-rmse:2.88689e+06+48350.4	test-rmse:7.40727e+06+104407
[138]	train-rmse:2.87686e+06+47077.4	test-rmse:7.41111e+06+105914
[139]	train-rmse:2.86867e+06+46471.7	test-rmse:7.41099e+06+103545
[140]	train-rmse:2.86391e+06+45361	test-rmse:7.41155e+06+103188
[141]	train-rmse:2.84955e+06+50790.7	test-rmse:7.41336e+06+104209
[142]	train-rm

[252]	train-rmse:2.16375e+06+59014.6	test-rmse:7.55146e+06+110244
[253]	train-rmse:2.16162e+06+59379.7	test-rmse:7.55193e+06+111003
[254]	train-rmse:2.15704e+06+60534.3	test-rmse:7.55178e+06+109588
[255]	train-rmse:2.15414e+06+60679.2	test-rmse:7.55373e+06+110321
[256]	train-rmse:2.14896e+06+60630	test-rmse:7.55489e+06+111011
[257]	train-rmse:2.14654e+06+60728.1	test-rmse:7.55557e+06+111594
[258]	train-rmse:2.14349e+06+62146.4	test-rmse:7.55616e+06+111115
[259]	train-rmse:2.13783e+06+60917.2	test-rmse:7.55821e+06+110839
[260]	train-rmse:2.13476e+06+60387.5	test-rmse:7.5593e+06+110089
[261]	train-rmse:2.12579e+06+60373.7	test-rmse:7.5612e+06+110595
[262]	train-rmse:2.12009e+06+61741.2	test-rmse:7.56151e+06+111277
[263]	train-rmse:2.11768e+06+62481.9	test-rmse:7.56300e+06+112130
[264]	train-rmse:2.11458e+06+63138.9	test-rmse:7.56365e+06+111998
[265]	train-rmse:2.11234e+06+63333.2	test-rmse:7.56375e+06+111774
[266]	train-rmse:2.10531e+06+65949.9	test-rmse:7.56509e+06+110383
[267]	train-rm

KeyboardInterrupt: 

## Train 

In [25]:
X_train_sp, X_val_sp, y_train_sp, y_val_sp = train_test_split(df_train, y_train, test_size = 0.2, random_state = 42)

In [26]:
tr_data = xgb.DMatrix(X_train_sp, y_train_sp)
va_data = xgb.DMatrix(X_val_sp, y_val_sp)
    
watchlist = [(tr_data, 'train'), (va_data, 'valid')]

In [29]:
xgb_regressor = xgb.train(params_xgb,
                          tr_data,
                          num_boost_round=num_trees,
                          evals=watchlist,
                          maximize=False,
                          early_stopping_rounds=early_stopping_rounds,
                          verbose_eval=verbose_eval
                          )

[0]	train-rmse:13.9627	valid-rmse:13.9519
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 30 rounds.
[10]	train-rmse:12.6509	valid-rmse:12.638
[20]	train-rmse:11.4653	valid-rmse:11.4513
[30]	train-rmse:10.3935	valid-rmse:10.3784
[40]	train-rmse:9.42697	valid-rmse:9.41246
[50]	train-rmse:8.55491	valid-rmse:8.54083
[60]	train-rmse:7.7681	valid-rmse:7.75486
[70]	train-rmse:7.05805	valid-rmse:7.0507
[80]	train-rmse:6.41737	valid-rmse:6.41471
[90]	train-rmse:5.8389	valid-rmse:5.84337
[100]	train-rmse:5.31751	valid-rmse:5.32993
[110]	train-rmse:4.84699	valid-rmse:4.86811
[120]	train-rmse:4.42403	valid-rmse:4.45285
[130]	train-rmse:4.04252	valid-rmse:4.08194
[140]	train-rmse:3.69795	valid-rmse:3.74945
[150]	train-rmse:3.38764	valid-rmse:3.45399
[160]	train-rmse:3.10902	valid-rmse:3.19113
[170]	train-rmse:2.85814	valid-rmse:2.95695
[180]	train-rmse:2.63241	valid-rmse:2.75001
[190]	train-rmse:2.42842	valid-rms

## Save model

In [31]:
model_name = 'baseline.xgb'
xgb_regressor.save_model(models_path + model_name)

## Predict

In [33]:
te_data = xgb.DMatrix(df_test)
y_pred_xgb = np.expm1(xgb_regressor.predict(te_data, ntree_limit=xgb_regressor.best_ntree_limit))

## Submit

In [37]:
submit_name = 'baseline.csv'

sub_xgb = pd.read_csv(raw_data_path+'sample_submission.csv')
sub_xgb["target"] = y_pred_xgb
sub_xgb.to_csv(submit_path+submit_name, index=False)

# Check 

In [38]:
sub_xgb.head()

,ID,target
0,000137c73,1616274.875
1,00021489f,1699441.625
2,0004d7953,3644900.750
3,00056a333,5574462.500
4,00056d8eb,1677115.125
